<a href="https://colab.research.google.com/github/TehLedRed/SpringerManuscript/blob/main/Simulation/NegativeBinomial/generateData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

def convertTheta__(x, theta): # = [0.1, 0.8, 1.5, 2.0, 2.5]
    if   x == 0:   return theta[0]
    elif x == 1:   return theta[1]
    elif x == 2:   return theta[2]
    elif x == 3:   return theta[3]
    else:          return theta[4]

def theta__(x, theta):
  f = lambda x: convertTheta__(x, theta)
  return np.vectorize(f)(x)

# theta__ = np.vectorize(convertTheta__, otypes=[float])

def samplingImg(labels, n, p1 = 0.1):

  i_1 = np.where(labels == 0)[0]
  i_2 = np.where(np.isin(labels, range(1,5)))[0]

  sample1 = np.random.choice(i_1, int(np.floor(n*p1)), replace=False)
  sample2 = np.random.choice(i_2, int(np.floor(n*(1-p1))), replace=False)

  index = np.concatenate((sample1, sample2)).astype(int)
  np.random.shuffle(index)
  return index

def y__(m):
    """
    Generate risk factors life time

    :param m: number of risk factors
    """
    if m == 0:
      return float('inf')
    else:
      return min(np.random.exponential(1, m))

def simulateData(labels, sampleIndex, phi, c_low = 0.1, theta = [0.1, 0.8, 1.5, 2.0, 2.5], poisson = False):

  n = len(sampleIndex)
  x = labels[sampleIndex]
  thetas = theta__(x, theta)

  # generate number of risk factors (M)
  if (poisson):
    M = np.random.poisson(lam = thetas, size = n)
  else:
    M = np.random.negative_binomial(n = 1/phi, p = 1/(1 + phi*thetas), size = n)

  # generate survival time (Y)
  data = pd.DataFrame(zip(sampleIndex, thetas, M),
                          columns = ["index", "theta", "m"])
  data['y'] = data['m'].map(y__)

  y1 = data[data["y"] != float('inf')]["y"]
  fig, ax = plt.subplots(figsize =(8,4))
  ax.hist(y1.values, bins =20)
  ax.set_title("Distribution of survival times")
  plt.figtext(0.5,0.5, y1.describe().to_string())
  plt.show()

  # generate censoring time (C)
  data['c'] = np.random.uniform(low = c_low, high = np.ceil(np.max(y1.values)) + 0.5, size = n)

  data = data.assign(t = lambda x: np.minimum(x.y.values, x.c.values))

  data['delta'] = np.where(data.t < data.c, 1, 0)
  # data['t'] = np.where(data.t > 3, 3, data.t)

  return(data)